In [2]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Prepare Batch

In [3]:
import os
import torch
import pandas as pd
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [4]:
recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [5]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [6]:
from fieldnn.sublayer.expander import Expander_Layer
from fieldnn.configfn.expanderfn import get_expander_para

full_recfldgrn_list = [
                  i for i in batch_rfg
                ]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [7]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    # no matter what type of grain, in the end, we will have vocab_tokenizer.
    if 'LLM' in full_recfldgrn:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = vocab_tokenizer.name_or_path
    else:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
    
len(full_rfg_to_basicInfo)

9

In [8]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


full_rfg_to_ExpanderConfig = {}

for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CateEmbed'

    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbed'

    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumeEmbed'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    
    # print('\n----')
    # print(full_recfldgrn)
    # print(d)
    # print(nn_name)
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
    
# full_rfg_to_ExpanderConfig

In [9]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])

B-P@age-AgeNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumeEmbeddingLayer(
    (embedding): Embedding(19, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P@basicInfo-basicInfoDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(7, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@BasicInfo-BasicDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(9, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@DT_min-DTDftGrn_i

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

In [10]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information


B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@DT-DTDftGrn_idx
(4, 25, 1, 7)
torch.Size([4, 25, 1, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@DT-DTDftGrn_idx
(4, 25, 22, 7)
torch.Size([4, 25, 22, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder sha

In [11]:
full_recfldgrn_EmbedTensor = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
    
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')
    full_recfldgrn_EmbedTensor[output_recfld] = {'holder': holder, 'info': info}
    print('\n')

Input: B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information
torch.Size([4, 19]) <----- holder shape
Output: B-P@age-AgeNumeDft
torch.Size([4, 19, 128]) <----- info shape


Input: B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 2]) <----- holder shape
Output: B-P@basicInfo-basicInfoDft
torch.Size([4, 2, 128]) <----- info shape


Input: B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 2]) <----- holder shape
Output: B-P-EC@BasicInfo-BasicDft
torch.Size([4, 25, 2, 128]) <----- info shape


Input: B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 7]) <----- holder shape
Output: B-P-EC@DT_min-DTDft
torch.Size([4, 25, 7, 128]) <----- info shape


Input: B-P

# Get the Starting Tensor and Tensor Name

In [12]:
# full_recfldgrn_EmbedTensor = {}
RECFLD_TO_TENSOR = {}

for full_recfld, info_dict in full_recfldgrn_EmbedTensor.items():
    print(full_recfld, info_dict['info'].shape)
    RECFLD_TO_TENSOR[full_recfld] = info_dict

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


In [13]:
[i for i in RECFLD_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [14]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_TENSOR])
max_layer

7

In [15]:
import numpy as np

def get_dataflow_table(full_recfldgrn_list):
    '''
        input: a list of full_recfldgrn
        output: the dataframe that shows the data flow from the grain-embedding-tensor to final feature vector.
    '''
    max_layer = max([len(i.split('-')) for i in full_recfldgrn_list])
    L = []
    for full_recfldgrn in full_recfldgrn_list:
        d = {}
        recfldgrn = '-'.join(full_recfldgrn.split('-')[-2:])
        current_layer_id = len(full_recfldgrn.split('-'))
        d['recfldgrn'] = recfldgrn
        for layer_id in range(max_layer, 1, -1):
            if layer_id > current_layer_id:
                d[layer_id] = np.nan
            elif layer_id == current_layer_id:
                d[layer_id] = full_recfldgrn
            else:
                d[layer_id] = np.nan
        L.append(d)
    df_dataflow = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow


def get_curfld_recinfo(curfld):
    '''
        a helper function that get the rec information from the current recfldgrn name.
        the input is: prefix_rec@fld_grn
        we want to check whether @ is here. 
        If there is a @ in the current level, we want to put the rec as the potential merger target. 
    '''
    if '@' in curfld:
        fld_list = curfld.split('@')
        return '@'.join(fld_list[:-1]), fld_list[-1]
    else:
        return None, curfld

In [16]:
def update_df_dataflow(df, style = 'Reducer&Merger'):
    
    df_dataflow = df.copy()

    for layer_idx in df_dataflow.columns:
        
        # (1) first deal with the fld in each layer
        s = df_dataflow[layer_idx]
        current_fullrfg_list = s[-s.isna()].to_list()
        fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
        fld_with_at_to_keep = []
        fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
        
        while len(fld_with_at_to_check) > 0:
            for index in df_dataflow.index:
                full_recfldgrn = df_dataflow.loc[index, layer_idx]
                if pd.isna(full_recfldgrn): continue
                if full_recfldgrn == 'ToFill': continue
                
                if '@' in full_recfldgrn.split('-')[-1]:
                    
                    prefix = '-'.join(full_recfldgrn.split('-')[:-1])
                    curfld = full_recfldgrn.split('-')[-1] # PNSectSent@Sentence@Tk
                    headfld, tailfld = get_curfld_recinfo(curfld) # head: PNSectSent@Sentence, tail: Tk
                    
                    full_recfldgrn_new = full_recfldgrn.replace('@' + tailfld, '')
                    same_rfg_new = [i for i in current_fullrfg_list if full_recfldgrn_new in i]
                    
                    # print(f'for Layer {layer_idx}')
                    # print(full_recfldgrn, '<--- full_recfldgrn')
                    # print(full_recfldgrn_new, '<--- full_recfldgrn_new')
                    # print(same_rfg_new, '<--- same_rfg_new')
                    
                    if len(same_rfg_new) == 1:
                        # print(full_recfldgrn)
                        df_dataflow.loc[index, layer_idx] = full_recfldgrn_new
                        ############# need to think about this
                        s = df_dataflow[layer_idx]
                        current_fullrfg_list = s[-s.isna()].to_list()
                        #############
                        

                    elif len(same_rfg_new) > 1:
                        
                        if style == 'Reducer&Merger':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                            # To update a new Column
                            tailfld_merged = '&'.join([i.replace(full_recfldgrn_new +'@', '') for i in same_rfg_new])
                            new_index = '(Merge)' + headfld + '@' + tailfld_merged#
                            if new_index in df_dataflow.index: continue

                            selected_index_list = [df_dataflow[df_dataflow[layer_idx] == i].index[0] for i in same_rfg_new]
                            # print(selected_index_list)

                            l = list(df_dataflow.index)
                            # print(l, '<--- l original columns')
                            loc = max([l.index(i) for i in selected_index_list])
                            l.insert(loc + 1, new_index)
                            # print(l, '<--- l with new index columns')

                            df_dataflow.loc[new_index, layer_idx] = full_recfldgrn_new
                            df_dataflow = df_dataflow.reindex(l)
                            # print(list(df_dataflow.index), '<--- df_dataflow with new index columns')

                            ############# need to think about this
                            # s = df_dataflow[layer_idx]
                            # current_fullrfg_list = s[-s.isna()].to_list()
                            #############
                            fld_with_at_to_keep.append(full_recfldgrn_new)
                            # print(fld_with_at_to_keep, '<----------------'
                            
                        elif style == 'ReducerOnly':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                        else:
                            raise ValueError('Wrong style, only for Reducer&Merger and ReducerOnly')
                            
            
            # (2) update conditions
            s = df_dataflow[layer_idx]
            current_fullrfg_list = s[-s.isna()].to_list()
            fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
            fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
            
        if layer_idx == df_dataflow.columns[-1]: continue
        # then update the next layer information
        for index in df_dataflow.index:
            last = df_dataflow.loc[index, layer_idx]
            if pd.isna(last): continue
            if '@' in last.split('-')[-1] and style == 'Reducer&Merger': continue
            full_recfldgrn_next = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
            df_dataflow.loc[index, layer_idx - 1] = full_recfldgrn_next
            
            
    if style == 'ReducerOnly':
        df_dataflow.loc['(Merge)P@All', 2] = 'B-P' 
        
    return df_dataflow

In [17]:
def update_df_dataflow_completename(df):
    df_dataflow = df.copy()
    L = []
    for recfldgrn, row in df_dataflow.iterrows():
        # print(recfldgrn)
        new_row = {}
        new_row['recfldgrn'] = recfldgrn
        full_recfldgrn = [i for i in row.values if not pd.isna(i)][0]
        curlayer_idx = len(full_recfldgrn.split('-'))
        for layer_idx, full_recfldgrn in row.items():
            if curlayer_idx < layer_idx:
                new_row[layer_idx] = np.nan
            elif curlayer_idx == layer_idx:
                new_row[layer_idx] = full_recfldgrn
            elif curlayer_idx > layer_idx:
                last = new_row[layer_idx+1]
                output_fullname = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
                new_row[layer_idx] = output_fullname
            else:
                raise ValueError('wrong information')
        L.append(new_row)
    df_dataflow_filled = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow_filled

In [18]:
full_recfldgrn_list = [i for i in RECFLD_TO_TENSOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [19]:
df_dataflow = get_dataflow_table(full_recfldgrn_list)
df_dataflow

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,NaN
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,NaN
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,NaN,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,NaN,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,NaN,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,NaN,NaN,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,NaN,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,NaN,NaN,NaN
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,NaN,NaN,NaN,NaN,NaN


In [20]:
df_dataflow_new = update_df_dataflow(df_dataflow, style = 'ReducerOnly')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,B-P@EC@BasicInfo
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,B-P@EC@DT_min
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,B-P-EC@A1C@DT,B-P@EC@A1C@DT
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,B-P-EC@A1C@V,B-P@EC@A1C@V
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,B-P-EC@Diag@DT,B-P@EC@Diag@DT
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,B-P-EC@Diag@Value,B-P@EC@Diag@Value
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk,B-P-EC-PN-PNSect-PNSectSent,B-P-EC-PN-PNSect,B-P-EC-PN,B-P-EC@PN,B-P@EC@PN


In [21]:
df_dataflow_new = update_df_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,NaN,NaN
(Merge)A1C@DT&V,NaN,NaN,NaN,B-P-EC-A1C,B-P-EC@A1C,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,NaN,NaN


# Get Layer Neural Name

## Developing Process (Can close this)

In [20]:
df_dataflow = df_dataflow_new.copy()

In [21]:
layeridx_list = list(df_dataflow.columns)
layeridx_list

[7, 6, 5, 4, 3, 2]

In [22]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    print(A_layerid, B_layerid)

7 6
6 5
5 4
4 3
3 2


In [23]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    # print(A_layerid, B_layerid)
    
    
    A_tensors = df_dataflow[A_layerid]
    A_tensors = A_tensors[-A_tensors.isna()].to_dict()
    
    B_tensors = df_dataflow[B_layerid]
    B_tensors = B_tensors[-B_tensors.isna()].to_dict()
    
    
    print(f'\nFrom Layer {A_layerid} to {B_layerid}:')
    
    # from A tensor to B tensor, we have the Reducer NNs.
    # also notice that some 
    print('A', A_tensors)

    
    # from B tensor: potential there are some Merger NNs. 
    print('B', B_tensors)
    
    


From Layer 7 to 6:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'}
B {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}

From Layer 6 to 5:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}
B {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}

From Layer 5 to 4:
A {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}
B {'EC@BasicInfo-BasicDft': 'B-P-EC@BasicInfo-BasicDft', 'EC@DT_min-DTDft': 'B-P-EC@DT_min-DTDft', 'A1C@DT-DTDft': 'B-P-EC-A1C@DT', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V', '(Merge)A1C@DT&V': 'B-P-EC-A1C', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT', 'Diag@Value-DiagDf

In [24]:
NN_List = []

for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    # print(A_layerid, B_layerid)
    
    
    A_tensors = df_dataflow[A_layerid]
    A_tensors = A_tensors[-A_tensors.isna()].to_dict()
    
    B_tensors = df_dataflow[B_layerid]
    B_tensors = B_tensors[-B_tensors.isna()].to_dict()
    
    
    print(f'\nFrom Layer {A_layerid} to {B_layerid}:')
    
    # from A tensor to B tensor, we have the Reducer NNs.
    # also notice that some 
    
    print('A', A_tensors)
    for k in A_tensors:
        if k not in B_tensors: continue
        
        input_name = A_tensors[k]
        output_name = B_tensors[k]
        
        if pd.isna(output_name) == True: continue # pass it. 
        
        d = {}
        d['SubUnitName'] = 'Reducer'
        d['input_names'] = [input_name]
        d['output_name'] = output_name
        d['input_layerid'] = A_layerid
        d['output_layerid'] = B_layerid
        NN_List.append(d)
        
        
    # from B tensor: potential there are some Merger NNs. 
    print('B', B_tensors)
    # check whether these is a '(Merger') in the key
    merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
    # print('B-merger_tensors', merger_tensors)
    
    for output_tensor in merger_tensors:
        input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
        # print(output_tensor, ':', input_tensors)
        
        d = {}
        d['SubUnitName'] = 'Merger'
        d['input_names'] = input_tensors
        d['output_name'] = output_tensor
        
        d['input_layerid'] = B_layerid
        d['output_layerid'] = B_layerid
        
        NN_List.append(d)
        
        

df_NN = pd.DataFrame(NN_List)
df_NN
    


From Layer 7 to 6:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'}
B {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}

From Layer 6 to 5:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}
B {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}

From Layer 5 to 4:
A {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}
B {'EC@BasicInfo-BasicDft': 'B-P-EC@BasicInfo-BasicDft', 'EC@DT_min-DTDft': 'B-P-EC@DT_min-DTDft', 'A1C@DT-DTDft': 'B-P-EC-A1C@DT', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V', '(Merge)A1C@DT&V': 'B-P-EC-A1C', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT', 'Diag@Value-DiagDf

,nntype,input_names,output_name,input_layerid,output_layerid
0,Reducer,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,Reducer,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,Reducer,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,Reducer,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,Reducer,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,Reducer,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,Reducer,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,Merger,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,Merger,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,Reducer,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


In [25]:
df_dataflow

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,NaN,NaN
(Merge)A1C@DT&V,NaN,NaN,NaN,B-P-EC-A1C,B-P-EC@A1C,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,NaN,NaN


##  Write Up a Function

In [46]:


def get_SubUnit_List(df_dataflow):
    layeridx_list = list(df_dataflow.columns)
    
    NN_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = 'R'
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
        
            NN_List.append(d)


        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # print('B-merger_tensors', merger_tensors)

        for output_tensor in merger_tensors:
            input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = 'MR'
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = B_layerid
            d['output_layerid'] = B_layerid
        
            NN_List.append(d)
            
    df_NN = pd.DataFrame(NN_List)
    return df_NN


In [47]:
df_SubUnit = get_SubUnit_List(df_dataflow)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3
